<img src="images/inmas.png" width=130x align='right' />

# Notebook 19 - Pandas

In this notebook, you will learn:
- basic reading and writing commands
- the concept of data frames
- manipulating amd cleaning data set with Pandas
- masking and filtering

### Prerequisite

Notebook 18


### Housekeeping

In [ ]:
import pandas as pd

### What is Pandas?
- Pandas is a Python package which offers data structures and operations for manipulating numerical tables and time series
- The term pandas is derived from "panel data," an econometrics term describing a particular type of dataset

### Reading data from files

- The primary way you'll likely use `pandas` is by exploring and manipulating data in tables
    - often these data are from data warehouses elsewhere
- Pandas can read and write files in a variety of data formats
    - common data formats include CSV and TSV files (comma and tab separated values)
    - data can also be imported from .json, Excel, SQL files, and more!

Let's read a file named 'cereals.csv' stored in the *data* folder of this workshop. This file contains the nutritive value of boxed cereals. Values are separated by semicolons.

In [ ]:
file = "data/cereals.csv" 
cerealsDf = pd.read_csv(file, sep=';')
type(cerealsDf)

Notice the DataFrame object returned. These objects have many associated methods for exploring the data:
* `head()`: Prints the first few lines
* `tail()`: Prints the last few lines
* `describe()`: Returns a short decription
* `info()`: Provides info about the data frame

* `shape`: Attribute storing the shape of the data frame

### Previewing with `head` and `describe` and `shape`
- Pandas uses the first row of the data set to determine the name of the columns
- When using Pandas' methods it is best not wrap these calls in the `print` function for nicer output:

In [ ]:
cerealsDf.head()

In [ ]:
cerealsDf.describe()

In [ ]:
cerealsDf.shape

### Data often need to be cleaned

- Often, there will be issues with your data that needs to be fixed before it can be used

- Pandas offers many tools to help, the first one being

If you go back to the `head` output, you'll notice that row 1 of the csv file contains the data type

We'll skip it using the `skiprows` argument (giving a list of lines to skip):

In [ ]:
cerealsDf = pd.read_csv(file, sep=';', skiprows=[1])
cerealsDf.head()

### Finding missing data
The `info` method is very useful to assess the quality of the data file:


In [ ]:
cerealsDf.info()

### Filling in missing data
We found missing data in two columns are missing data: 'mfr' and 'rating'.
- For the 9 rows missing manufacturer, we'll replace missing values by 'U' (for Unknown)
- This is done using the `fillna` method which takes a dictionary as an argument

This fills missing values in the `mfr` column with the 'U' string

In [ ]:
cerealsDf = cerealsDf.fillna({'mfr': 'U'})

Running `info()` again, we see that the 'mfr' column is full:

In [ ]:
cerealsDf.info()

### Dropping missing data
- For the 7 missing 'rating' values, there's no obvious substitution, so we'll drop those entries
- Notice the `inplace` argument that prevent copying the dataframe

In [ ]:
cerealsDf.dropna(inplace=True)
cerealsDf.shape

We now have 70 rows left

Let's check info again...

In [ ]:
cerealsDf.info()

### About in-place operations

Pandas has many operations that can be performed *in-place*

- When *in-place*, the calling method directly alters the existing object and nothing is returned

- In contrast, "normal" operations make a copy of the object, alter that object, and then return the altered object. 

- Methods `fillna()`, `dropna()`, and many others accept the boolean argument `inplace`

Let's look at an example to highlight this behavior

### Demonstrating the in-place behavior
Let's start with making a copy of our data frame from which we'll make changes

In [ ]:
cerealsDfCopy = cerealsDf.copy()
# Drop the weight column NOT in-place
cerealsDfCopy.drop(["weight"], axis = 1, inplace = False)
cerealsDfCopy.head(3)

Notice that ```cerealsDFCcopy``` still has the column ```weight```. This is because the function did not alter the dataframe directly. Instead, a value was returned that had the altered data frame.

### Assigning altered data frame
We now repeat the same operation but assign the value returned by `drop`:

In [ ]:
cerealsDfCopyDrop = cerealsDfCopy.drop(["weight"], axis = 1, inplace = False)
cerealsDfCopyDrop.head(3)

Now compare this with the in-place version of the function, where we drop 'shelf' in-place instead

In [ ]:
cerealsDfCopy.drop(["shelf"], axis = 1, inplace = True)
cerealsDfCopy.head(3)

### Assigning an in-place change

Now what if one uses in-place and assigns the value?

In [ ]:
cerealsDfCopyDrop = cerealsDfCopy.drop(["cups"], axis = 1, inplace = True)
cerealsDfCopyDrop.head(3)

What went wrong?

###  Replacing values in named columns
There are several allowed input formats to `replace()` entries in a column:


In [ ]:
# Replace the letters in the `mfr` column with the actual names of the brands
# Using a simple dictionary and a string
cerealsDf.replace({"mfr": "N"}, "Nabisco", inplace = True)
# Using a dictionary of column and dictionaries
cerealsDf = cerealsDf.replace({"mfr" : {"Q" : "Quaker"}})

cerealsDf.head(5)

### Replacing column names
The `replace()` method can also be used to change the name of columns:

In [ ]:
cerealsDf.rename(columns={"protein": "protein (g)", "sodium": "sodium (mg)"}, inplace = True)
cerealsDf.head(3)

### Extracting rows and columns from a data frame
Extracting a column from a dataframe is very intuitive:

In [ ]:
fiber = cerealsDf["fiber"]
print(fiber)

### Using data frames and NumPy
We can easily convert a column to a NumPy array

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fiber = cerealsDf["fiber"]
fiber_array = fiber.to_numpy()

plt.hist(fiber_array);

### Extracting rows by index
We can extract a row using its index using the `loc` attribute

In [ ]:
row_2 = cerealsDf.loc[2] # 2 is the index of the row we want to extract
print(row_2)

### Extracting multiple rows and columns
To extract multiple rows and columns, we provide a list of values as an index

In [ ]:
mycols = ["mfr", "fiber"]
myrows = [1, 3, 5,20]
subset = cerealsDf[mycols].loc[myrows]
print(subset)

### Sorting
Data frames can be easily sorted:

In [ ]:
cerealsDf.sort_values(by=["rating"], ascending = False, inplace = True)
cerealsDf.head(5)

### Filtering
To filter, we can create a "mask", an array of the same size as our object, with entries `True` or `False` depending on whether a certain condition is met

In [ ]:
mask = (cerealsDf["calories"] > 90)
mask

Then we can use the mask to filter the dataframe:

In [ ]:
cereals_high_calories = cerealsDf[mask]
cereals_high_calories.head()

Masking can also be done directly:

In [ ]:
cereals_high_fiber = cerealsDf[(cerealsDf["fiber"] > 5)]
cereals_high_fiber.head(6)

While head is requesting 6 values, only 4 cereals have more that 5 g of fiber

### Key Points
- Pandas is a powerful libray for manipulating data
- It has many functions to clean the data by replacing or dropping entries
- Pandas data type integrate very well with the NumPy library and can be used in matplotlib
- Sorting can be done by column or rows
- Filtering can be done with masking

### Futher Reading
- Pandas documentation is [here](https://pandas.pydata.org/docs/index.html)
